In [30]:
import os
import importlib

import numpy as np
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import skimage

import torch
import torch.nn as nn

import load_data
#import first_exp_model_with_pytorch as dloader
#import first_exp_model_with_pytorch

In [2]:
plt.style.use('Solarize_Light2')
%matplotlib inline
importlib.reload(load_data)

<module 'load_data' from '/home/uldo/work/kaggle/competitions/Human_Protein_Atlas_Image_Classification/code/load_data.py'>

In [3]:
LABEL_NAMES_MAP = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}

In [26]:
def show(image, labels):
    f, (ax1,ax2,ax3,ax4) = plt.subplots(1, 4, figsize=(25,25), sharey=True)

    title = ''
    #npimage = image.numpy()[:, ::-1, :, :]
    print(image.size())
    print(labels)
    print(labels.size())
    #print(image)
    npimage = image.numpy()
    nplabels = labels.numpy()
    print(npimage.shape)
    print(nplabels)
    npimage_swap = npimage[:, ::-1]
    print(npimage_swap.shape)
    '''
    #labels =sample['target']
                
    #for i, label in enumerate(LABELS):
    #    if labels[i] == 1:
    #        if title == '':
    #            title += label
    #        else:
    #            title += " & " + label
            
    ax1.imshow(npimage[0,:,:],cmap="hot")
    ax1.set_title('Red')
    ax2.imshow(npimage[1,:,:],cmap="copper")
    ax2.set_title('Green')
    ax3.imshow(npimage[2,:,:],cmap="bone")
    ax3.set_title('Blue')
    ax4.imshow(npimage[3,:,:],cmap="afmhot")
    ax4.set_title('Yellow')
    f.suptitle(title, fontsize=20, y=0.62)
    '''

In [31]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(4, 8, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.drop_out = nn.Dropout()
        self.out1 = nn.Linear(int(16 * W/4 * H/4), 900)
        self.out2 = nn.Linear(900, 28)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.drop_out(x)
        output = self.out1(x)
        output = self.out2(output)
        return output, x

In [5]:
train_df = load_data.load_text_data('../input/train.csv')

In [6]:
#train_dataset = dloader.HumanProteinAtlasDataset(train_df, transform=dloader.transforms.ToTensor)
train_dataset = load_data.HumanProteinAtlasDataset(
    train_df,
    transform=load_data.transforms.ToTensor()
)

In [7]:
#train_loader = dloader.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)
train_loader = load_data.data.DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=1)

In [8]:
print(train_loader)

In [9]:
train_iter = iter(train_loader)

In [10]:
print(type(train_iter))

<class 'torch.utils.data.dataloader._DataLoaderIter'>


In [11]:
images, labels = train_iter.next()

In [12]:
print("images shape on batch size = {}".format(images.size()))
print("labels shape on batch size = {}".format(labels.size()))

images shape on batch size = torch.Size([10, 4, 512, 512])
labels shape on batch size = torch.Size([10, 1, 28])


In [13]:
#import torchvision

In [14]:
#grid = torchvision.utils.make_grid(images)
#grid = torchvision.utils.make_grid([image[0] for image in images])

In [15]:
#plt.imshow(grid)
#plt.axis('off')
#plt.title(labels.numpy())

In [28]:
#show(images[0], labels[0])